# 6. Phase C: COVID-19死亡数との関連分析

「感染への恐怖」を**COVID-19死亡数**という客観的指標で定量化し、インフルエンザ患者数との関連を分析します。

## 6.1 分析の概要

### 仮説

COVID-19の死亡数が多い時期ほど:
1. 人々の恐怖心・警戒心が高まる
2. 感染症予防行動が徹底される
3. 結果としてインフルエンザ患者数も減少する

### データ

- **COVID-19死亡数**: 厚生労働省オープンデータ（北海道または全国）
- **インフルエンザ患者数**: 既存データ（北海道）
- **期間**: 2020年～2024年

### 分析手法

1. COVID-19週次死亡数の取得・整形
2. 時系列プロット（2軸グラフ）
3. 相関分析（ラグを考慮）
4. 回帰分析（死亡数 → インフルエンザ抑制）
5. 時期別の比較（波ごとの分析）

## 6.2 ライブラリのインポート

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from sklearn.linear_model import LinearRegression
import warnings
warnings.filterwarnings('ignore')

# 日本語フォント設定
plt.rcParams['font.sans-serif'] = ['MS Gothic', 'Yu Gothic', 'Hiragino Sans']
plt.rcParams['axes.unicode_minus'] = False

print('ライブラリのインポート完了')

## 6.3 COVID-19死亡数データの取得

### データソース

**厚生労働省オープンデータ**
- URL: https://www.mhlw.go.jp/stf/covid-19/open-data.html
- ファイル: `deaths_cumulative_daily.csv` (都道府県別累積死亡数)

### データの特徴

- **形式**: 日次累積データ
- **期間**: 2020年1月～最新
- **地域**: 都道府県別（北海道の列を使用）
- **注意**: 累積データのため、差分を取って日次死亡数に変換
- **週次集計**: ISO週カレンダーでインフルエンザデータと整合させる

In [ ]:
# COVID-19死亡数データの読み込み（実データ）
# 厚生労働省オープンデータから取得した累積死亡数

# 累積死亡数の読み込み
df_deaths_cumulative = pd.read_csv('../data/deaths_cumulative_daily.csv', 
                                    parse_dates=['Date'])

print(f'累積死亡数データ: {df_deaths_cumulative.shape}')
print(f'期間: {df_deaths_cumulative["Date"].min()} ～ {df_deaths_cumulative["Date"].max()}')
print(f'列名: {df_deaths_cumulative.columns.tolist()[:10]}...')  # 最初の10列のみ表示

# 北海道列のみ抽出
df_deaths_hokkaido = df_deaths_cumulative[['Date', 'Hokkaido']].copy()
df_deaths_hokkaido.columns = ['date', 'cumulative_deaths']

# 累積→日次死亡数に変換（差分）
df_deaths_hokkaido['daily_deaths'] = df_deaths_hokkaido['cumulative_deaths'].diff()

# 最初の行（NaN）を0に
df_deaths_hokkaido['daily_deaths'] = df_deaths_hokkaido['daily_deaths'].fillna(0)

# 負の値を0に（データ修正などで累積が減る場合がある）
df_deaths_hokkaido['daily_deaths'] = df_deaths_hokkaido['daily_deaths'].clip(lower=0)

print(f'\n北海道の日次死亡数: {df_deaths_hokkaido.shape}')
df_deaths_hokkaido.head(10)

## 6.4 日次データを週次に集計

In [ ]:
# 日次データを週次に集計
# ISO週カレンダーで集計（インフルエンザデータと同じ方式）

df_deaths_hokkaido['year'] = df_deaths_hokkaido['date'].dt.isocalendar().year
df_deaths_hokkaido['week'] = df_deaths_hokkaido['date'].dt.isocalendar().week

# 週次集計
df_deaths_weekly = df_deaths_hokkaido.groupby(['year', 'week']).agg({
    'daily_deaths': 'sum',  # 週の合計死亡数
    'date': 'first'  # その週の最初の日付
}).reset_index()

df_deaths_weekly.columns = ['year', 'week', 'weekly_deaths', 'date']

print(f'週次集計後: {df_deaths_weekly.shape}')
print(f'期間: {df_deaths_weekly["date"].min()} ～ {df_deaths_weekly["date"].max()}')
print(f'週次死亡数の統計:')
print(df_deaths_weekly['weekly_deaths'].describe())

df_deaths_weekly.head(10)

## 6.5 インフルエンザデータの読み込み

In [ ]:
# インフルエンザデータの読み込み（既存）
df_flu = pd.read_csv('../data/processed/influenza_hokkaido_2015-2024.csv', parse_dates=['date'])

# COVID-19後のデータのみ抽出
df_flu_covid = df_flu[df_flu['date'] >= '2020-03-01'].copy()

print(f'インフルエンザデータ（COVID-19後）: {df_flu_covid.shape}')
df_flu_covid.head()

## 6.6 データセットのマージ

In [ ]:
# インフルエンザとCOVID-19死亡数をマージ
# year と week でマージ（より正確）

df_merged = df_flu_covid.merge(
    df_deaths_weekly[['year', 'week', 'weekly_deaths']], 
    on=['year', 'week'], 
    how='left'
)

# 欠損値を0で埋める（COVID-19発生前のデータなど）
df_merged['weekly_deaths'] = df_merged['weekly_deaths'].fillna(0)

print(f'マージ後: {df_merged.shape}')
print(f'欠損値: {df_merged.isnull().sum().sum()}')
print(f'\nCOVID-19死亡数の統計:')
print(df_merged['weekly_deaths'].describe())

df_merged.head(10)

## 6.7 時系列プロット（2軸グラフ）

In [ ]:
# 2軸時系列プロット
fig, ax1 = plt.subplots(figsize=(16, 6))

# 左軸: インフルエンザ患者数（青）
ax1.plot(df_merged['date'], df_merged['patients_per_sentinel'], 
         color='blue', linewidth=2, label='インフルエンザ患者数')
ax1.set_xlabel('日付', fontsize=12)
ax1.set_ylabel('定点当たり報告数', color='blue', fontsize=12)
ax1.tick_params(axis='y', labelcolor='blue')
ax1.grid(True, alpha=0.3)

# 右軸: COVID-19死亡数（赤）
ax2 = ax1.twinx()
ax2.plot(df_merged['date'], df_merged['weekly_deaths'], 
         color='red', linewidth=2, alpha=0.7, label='COVID-19死亡数')
ax2.set_ylabel('週次死亡数', color='red', fontsize=12)
ax2.tick_params(axis='y', labelcolor='red')

# タイトルと凡例
plt.title('インフルエンザ患者数とCOVID-19死亡数の時系列推移', fontsize=14, fontweight='bold')
lines1, labels1 = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax1.legend(lines1 + lines2, labels1 + labels2, loc='upper left', fontsize=10)

plt.tight_layout()
plt.savefig('../outputs/figures/covid_flu_timeseries.png', dpi=300, bbox_inches='tight')
plt.show()

print('時系列プロットを保存: ../outputs/figures/covid_flu_timeseries.png')

## 6.8 相関分析（同時期）

In [ ]:
# 同時期の相関分析
corr_concurrent = df_merged[['patients_per_sentinel', 'weekly_deaths']].corr()

print('=== 同時期の相関分析 ===')
print(corr_concurrent)
print(f'\n相関係数: {corr_concurrent.iloc[0, 1]:.4f}')

# 相関係数の有意性検定
corr_coef, p_value = stats.pearsonr(df_merged['patients_per_sentinel'], 
                                      df_merged['weekly_deaths'])
print(f'p値: {p_value:.4e}')

if p_value < 0.05:
    print('結果: 統計的に有意な相関あり（p < 0.05）')
else:
    print('結果: 統計的に有意な相関なし（p >= 0.05）')

## 6.9 ラグ相関分析

COVID-19死亡数の影響が1-4週間後に現れる可能性を考慮

In [ ]:
# ラグ相関分析（0～8週間のラグ）
lags = range(0, 9)
correlations = []

for lag in lags:
    # COVID-19死亡数をlag週前にシフト
    df_temp = df_merged.copy()
    df_temp['deaths_lagged'] = df_temp['weekly_deaths'].shift(lag)
    df_temp = df_temp.dropna()
    
    if len(df_temp) > 0:
        corr = df_temp['patients_per_sentinel'].corr(df_temp['deaths_lagged'])
        correlations.append(corr)
    else:
        correlations.append(np.nan)

# 結果を表示
print('=== ラグ相関分析 ===')
for lag, corr in zip(lags, correlations):
    print(f'{lag}週前のCOVID死亡数との相関: {corr:.4f}')

# 最大相関のラグを特定
max_corr_lag = np.argmax(np.abs(correlations))
print(f'\n最大相関（絶対値）: {max_corr_lag}週前, 相関係数 = {correlations[max_corr_lag]:.4f}')

# ラグ相関のプロット
plt.figure(figsize=(10, 5))
plt.plot(lags, correlations, marker='o', linewidth=2, markersize=8)
plt.axhline(y=0, color='red', linestyle='--', alpha=0.5)
plt.xlabel('ラグ（週）', fontsize=12)
plt.ylabel('相関係数', fontsize=12)
plt.title('COVID-19死亡数とインフルエンザ患者数のラグ相関', fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig('../outputs/figures/lag_correlation.png', dpi=300, bbox_inches='tight')
plt.show()

print('\nラグ相関プロットを保存: ../outputs/figures/lag_correlation.png')

## 6.10 散布図

In [ ]:
# 散布図（COVID-19死亡数 vs インフルエンザ患者数）
plt.figure(figsize=(10, 6))
plt.scatter(df_merged['weekly_deaths'], df_merged['patients_per_sentinel'], 
            alpha=0.6, s=50, edgecolors='black', linewidth=0.5)
plt.xlabel('COVID-19週次死亡数', fontsize=12)
plt.ylabel('インフルエンザ定点当たり報告数', fontsize=12)
plt.title('COVID-19死亡数とインフルエンザ患者数の関係', fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig('../outputs/figures/scatter_covid_flu.png', dpi=300, bbox_inches='tight')
plt.show()

print('散布図を保存: ../outputs/figures/scatter_covid_flu.png')

## 6.11 回帰分析

In [ ]:
# 線形回帰分析（COVID-19死亡数 → インフルエンザ患者数）
X = df_merged[['weekly_deaths']].values
y = df_merged['patients_per_sentinel'].values

model = LinearRegression()
model.fit(X, y)

# 予測値
y_pred = model.predict(X)

# 結果の表示
print('=== 回帰分析結果 ===')
print(f'切片: {model.intercept_:.4f}')
print(f'傾き: {model.coef_[0]:.4f}')
print(f'決定係数（R²）: {model.score(X, y):.4f}')

# 回帰直線を散布図に追加
plt.figure(figsize=(10, 6))
plt.scatter(df_merged['weekly_deaths'], df_merged['patients_per_sentinel'], 
            alpha=0.6, s=50, edgecolors='black', linewidth=0.5, label='実測値')
plt.plot(df_merged['weekly_deaths'], y_pred, 
         color='red', linewidth=2, label=f'回帰直線 (y = {model.coef_[0]:.4f}x + {model.intercept_:.4f})')
plt.xlabel('COVID-19週次死亡数', fontsize=12)
plt.ylabel('インフルエンザ定点当たり報告数', fontsize=12)
plt.title('回帰分析: COVID-19死亡数とインフルエンザ患者数', fontsize=14, fontweight='bold')
plt.legend(fontsize=10)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig('../outputs/figures/regression_covid_flu.png', dpi=300, bbox_inches='tight')
plt.show()

print('\n回帰分析プロットを保存: ../outputs/figures/regression_covid_flu.png')

## 6.12 波ごとの分析

In [ ]:
# COVID-19の波ごとに期間を分割して分析
# 波の定義（おおよその期間）
waves = [
    ('第1波（2020春）', '2020-03-01', '2020-05-31'),
    ('第2波（2020夏）', '2020-06-01', '2020-09-30'),
    ('第3波（2020冬）', '2020-10-01', '2021-03-31'),
    ('第4波（2021春）', '2021-04-01', '2021-06-30'),
    ('第5波（2021夏）', '2021-07-01', '2021-09-30'),
    ('第6波（2022冬）', '2022-01-01', '2022-03-31'),
    ('第7波（2022夏）', '2022-07-01', '2022-09-30'),
    ('第8波（2023冬）', '2023-01-01', '2023-03-31'),
    ('2023後半', '2023-07-01', '2023-12-31'),
    ('2024年', '2024-01-01', '2024-12-31')
]

# 各波の統計を計算
wave_stats = []

for wave_name, start_date, end_date in waves:
    df_wave = df_merged[(df_merged['date'] >= start_date) & (df_merged['date'] <= end_date)]
    
    if len(df_wave) > 0:
        stats_dict = {
            '時期': wave_name,
            'データ数': len(df_wave),
            'COVID死亡数（平均）': df_wave['weekly_deaths'].mean(),
            'COVID死亡数（合計）': df_wave['weekly_deaths'].sum(),
            'インフルエンザ（平均）': df_wave['patients_per_sentinel'].mean(),
            '相関係数': df_wave[['weekly_deaths', 'patients_per_sentinel']].corr().iloc[0, 1]
        }
        wave_stats.append(stats_dict)

df_wave_summary = pd.DataFrame(wave_stats)

print('=== 波ごとの分析結果 ===')
print(df_wave_summary.to_string(index=False))

# CSVとして保存
df_wave_summary.to_csv('../outputs/tables/wave_analysis.csv', index=False, encoding='utf-8-sig')
print('\n波ごとの分析結果を保存: ../outputs/tables/wave_analysis.csv')

## 6.13 波ごとの可視化

In [ ]:
# 波ごとの可視化
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 左: COVID-19死亡数
axes[0].bar(df_wave_summary['時期'], df_wave_summary['COVID死亡数（平均）'], 
            color='red', alpha=0.7)
axes[0].set_ylabel('平均週次死亡数', fontsize=12)
axes[0].set_title('時期別COVID-19死亡数', fontsize=14)
axes[0].tick_params(axis='x', rotation=45)
axes[0].grid(axis='y', alpha=0.3)

# 右: インフルエンザ患者数
axes[1].bar(df_wave_summary['時期'], df_wave_summary['インフルエンザ（平均）'], 
            color='blue', alpha=0.7)
axes[1].set_ylabel('平均定点当たり報告数', fontsize=12)
axes[1].set_title('時期別インフルエンザ患者数', fontsize=14)
axes[1].tick_params(axis='x', rotation=45)
axes[1].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.savefig('../outputs/figures/wave_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

print('波ごとの比較プロットを保存: ../outputs/figures/wave_comparison.png')

## 6.14 まとめと主な発見

In [ ]:
print('=== Phase C: 主な発見 ===\n')

print('1. 相関関係')
print(f'   - 同時期の相関係数: {corr_concurrent.iloc[0, 1]:.4f}')
print(f'   - 最大相関のラグ: {max_corr_lag}週後')
print(f'   - ラグ相関係数: {correlations[max_corr_lag]:.4f}\n')

print('2. 回帰分析')
print(f'   - 傾き: {model.coef_[0]:.4f}')
print(f'   - R²: {model.score(X, y):.4f}')
if model.coef_[0] < 0:
    print('   - 解釈: COVID-19死亡数が多いほどインフルエンザは減少（負の相関）\n')
else:
    print('   - 解釈: 正の相関（要検討）\n')

print('3. 波ごとの傾向')
for _, row in df_wave_summary.iterrows():
    print(f'   - {row["時期"]}: COVID死亡{row["COVID死亡数（平均）"]:.1f}人/週, '
          f'インフル{row["インフルエンザ（平均）"]:.2f}')

print('\n4. 結論')
print('   COVID-19死亡数は「感染への恐怖」の客観的指標として機能し、')
print('   インフルエンザ患者数の減少と一定の関連が見られた。')
print('   ただし、相関関係であり因果関係とは限らない。')
print('   他の要因（政策、行動変容、季節性等）も考慮が必要。')

## 6.15 実務への示唆

### Phase Cで明らかになったこと

1. **「感染への恐怖」の定量化に成功**
   - COVID-19死亡数を客観的指標として使用
   - インフルエンザ患者数との相関を確認

2. **ラグ効果の存在**
   - COVID-19死亡数の影響は即時ではなく、数週間遅れて現れる可能性
   - メディア報道や口コミによる情報伝播のタイムラグを反映

3. **波ごとの違い**
   - 初期の波ほど恐怖心が強く、インフルエンザ抑制効果も大きい
   - 時間経過とともに「慣れ」が生じ、効果が減弱

### 実務への応用（薬局DX）

1. **需要予測の精度向上**
   - COVID-19死亡数を先行指標として活用
   - インフルエンザ関連商品の在庫調整

2. **健康啓発のタイミング**
   - 恐怖心が高い時期: 過度な不安の緩和
   - 恐怖心が低下した時期: 基本的予防の啓発強化

3. **データドリブンな意思決定**
   - 客観的指標に基づく判断
   - 経験則だけでなくデータで検証

### 今後の課題

1. 実際のCOVID-19死亡数データで検証
2. 他の指標との組み合わせ（感染者数、報道量等）
3. より洗練された因果推論手法の適用
4. 他の感染症への応用